<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
    return results["recip_rank"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[58.107510603567086, 'False', 0.071968468695619]                                                                       
,                                                                                                                      
[53.40237754126514, 'True', 0.07595842699411386]                                                                       
,                                                                                                                      
[21.84274566358853, 'True', 0.08877480938767326]                                                                       
,                                                                                                                      
[83.9315217221609, 'False', 0.0689222660200006]                                                                        
,                                                                                                                      
[99.31122572157601, 'True', 0.0623092909

[58.40197777204613, 'False', 0.07051066112262004]                                                                      
,                                                                                                                      
[8.233196122263585, 'True', 0.10072068581287306]                                                                       
,                                                                                                                      
[31.550088988729687, 'False', 0.08732561481704534]                                                                     
,                                                                                                                      
[18.60223713363755, 'True', 0.09702108945876206]                                                                       
,                                                                                                                      
[68.04321057333729, 'True', 0.0675532721

[5.828467108661435, 'True', 0.09635888757909802]                                                                       
,                                                                                                                      
[10.592689562201395, 'True', 0.10327770514606813]                                                                      
,                                                                                                                      
[20.620881726869356, 'True', 0.090385893904951]                                                                        
,                                                                                                                      
[24.56744546111185, 'True', 0.08730242471954144]                                                                       
,                                                                                                                      
[34.78779167561472, 'True', 0.0849837392

In [4]:
array = [[58.107510603567086, 'False', 0.071968468695619]                                                                       
,                                                                                                                      
[53.40237754126514, 'True', 0.07595842699411386]                                                                       
,                                                                                                                      
[21.84274566358853, 'True', 0.08877480938767326]                                                                       
,                                                                                                                      
[83.9315217221609, 'False', 0.0689222660200006]                                                                        
,                                                                                                                      
[99.31122572157601, 'True', 0.062309290937595414]                                                                      
,                                                                                                                      
[61.27269835261429, 'False', 0.07416729358659144]                                                                      
,                                                                                                                      
[65.04866095216828, 'False', 0.07420802563250467]                                                                      
,                                                                                                                      
[96.5986353273463, 'True', 0.06605985411572636]                                                                        
,                                                                                                                      
[81.64485732331923, 'True', 0.06865686106612959]                                                                       
,                                                                                                                      
[73.37995348343088, 'True', 0.06956777483315958]                                                                       
,                                                                                                                      
[13.37317433809126, 'True', 0.0971815661330045]                                                                        
,                                                                                                                      
[2.7678281329670824, 'False', 0.073553606356315]                                                                       
,                                                                                                                      
[22.135159180798865, 'True', 0.08948662624651998]                                                                      
,                                                                                                                      
[23.70592602200753, 'True', 0.08878076811425893]                                                                       
,                                                                                                                      
[12.490011524958886, 'True', 0.09388263609767786]                                                                      
,                                                                                                                      
[37.64564659967027, 'True', 0.08214037264326364]                                                                       
,                                                                                                                      
[82.51020740809199, 'True', 0.06703901669116435]                                                                       
,                                                                                                                      
[26.639978260638685, 'True', 0.09063874783844154]                                                                      
,                                                                                                                      
[90.72291235832414, 'False', 0.07093973952056663]                                                                      
,                                                                                                                      
[16.03145229948564, 'True', 0.0927061832337947]                                                                        
,                                                                                                                      
[5.447086549982089, 'True', 0.10054820922493322]                                                                       
,                                                                                                                      
[1.6756643179093125, 'True', 0.04802205510920909]                                                                      
,                                                                                                                      
[39.56974024280981, 'True', 0.0830240310608405]                                                                        
,                                                                                                                      
[8.515980115952097, 'True', 0.101761598537673]                                                                         
,                                                                                                                      
[37.56664371256538, 'True', 0.07863628209941143]                                                                       
,                                                                                                                      
[1.0351173465842702, 'True', 0.05030900588276065]                                                                      
,                                                                                                                      
[8.760154051951673, 'True', 0.10164917281719052]                                                                       
,                                                                                                                      
[44.65158943782709, 'True', 0.07779841146731757]                                                                       
,                                                                                                                      
[29.979108826424856, 'False', 0.08373667709869512]                                                                     
,                                                                                                                      
[9.11303929907589, 'True', 0.10147590600496335]                                                                        
,                                                                                                                      
[51.8239867414934, 'False', 0.07171555078407912]                                                                       
,                                                                                                                      
[31.382919666967314, 'True', 0.08320910413945498]                                                                      
,                                                                                                                      
[17.897687709433143, 'True', 0.0950705933150424]                                                                       
,                                                                                                                      
[44.0711541343798, 'True', 0.07639783357410015]                                                                        
,                                                                                                                      
[58.40197777204613, 'False', 0.07051066112262004]                                                                      
,                                                                                                                      
[8.233196122263585, 'True', 0.10072068581287306]                                                                       
,                                                                                                                      
[31.550088988729687, 'False', 0.08732561481704534]                                                                     
,                                                                                                                      
[18.60223713363755, 'True', 0.09702108945876206]                                                                       
,                                                                                                                      
[68.04321057333729, 'True', 0.06755327217463047]                                                                       
,                                                                                                                      
[8.248575998867596, 'True', 0.10015158422009278]                                                                       
,                                                                                                                      
[49.25577630879856, 'False', 0.07483018283345647]                                                                      
,                                                                                                                      
[26.435402575868835, 'True', 0.08699929840857878]                                                                      
,                                                                                                                      
[13.372416741309378, 'True', 0.09690012781395212]                                                                      
,                                                                                                                      
[68.48502533910123, 'False', 0.07199187788916513]                                                                      
,                                                                                                                      
[74.94162144859531, 'True', 0.07255991546788376]                                                                       
,                                                                                                                      
[4.572121424890901, 'True', 0.09310904010500222]                                                                       
,                                                                                                                      
[55.71004672469063, 'True', 0.0753013467505082]                                                                        
,                                                                                                                      
[21.525002042747488, 'True', 0.08661423438343431]                                                                      
,                                                                                                                      
[35.71454452981939, 'False', 0.07950986645434471]                                                                      
,                                                                                                                      
[11.663073818678718, 'True', 0.10048248403106459]                                                                      
,                                                                                                                      
[97.14460305060067, 'True', 0.06545890079905382]                                                                       
,                                                                                                                      
[48.55813526908589, 'True', 0.07518931507583687]                                                                       
,                                                                                                                      
[24.84966855545834, 'False', 0.08632954142814644]                                                                      
,                                                                                                                      
[20.94619607090561, 'True', 0.09155688718400652]                                                                       
,                                                                                                                      
[87.52907060139461, 'True', 0.07089527599586556]                                                                       
,                                                                                                                      
[77.66249576283312, 'True', 0.06846619276179718]                                                                       
,                                                                                                                      
[41.79894338396778, 'True', 0.07972952139300818]                                                                       
,                                                                                                                      
[33.6019094694997, 'False', 0.08398938968980603]                                                                       
,                                                                                                                      
[1.0888383825702066, 'True', 0.0485232143537991]                                                                       
,                                                                                                                      
[28.429132524196852, 'True', 0.0871598764951868]                                                                       
,                                                                                                                      
[4.618858061430882, 'True', 0.08743006934399011]                                                                       
,                                                                                                                      
[15.074445376165722, 'False', 0.09570614933497522]                                                                     
,                                                                                                                      
[18.241165343063763, 'True', 0.09380945189070032]                                                                      
,                                                                                                                      
[8.732420206736089, 'True', 0.10378538215498864]                                                                       
,                                                                                                                      
[11.408894111414856, 'True', 0.09926855903374594]                                                                      
,                                                                                                                      
[8.144113896311957, 'True', 0.10009674378105884]                                                                       
,                                                                                                                      
[15.921735456716892, 'True', 0.09494269714498003]                                                                      
,                                                                                                                      
[3.0550351956044084, 'True', 0.08633974318832102]                                                                      
,                                                                                                                      
[5.828467108661435, 'True', 0.09635888757909802]                                                                       
,                                                                                                                      
[10.592689562201395, 'True', 0.10327770514606813]                                                                      
,                                                                                                                      
[20.620881726869356, 'True', 0.090385893904951]                                                                        
,                                                                                                                      
[24.56744546111185, 'True', 0.08730242471954144]                                                                       
,                                                                                                                      
[34.78779167561472, 'True', 0.084983739282962]                                                                         
,                                                                                                                      
[14.394287182125918, 'True', 0.1020991235471801]                                                                       
,                                                                                                                      
[28.578080361345933, 'True', 0.0875213262381004]                                                                       
,                                                                                                                      
[39.23625331205547, 'True', 0.08366243916593341]                                                                       
,                                                                                                                      
[44.87803066186765, 'True', 0.07770008736752111]                                                                       
,                                                                                                                      
[60.57229627664713, 'True', 0.07095116722526265]                                                                       
,                                                                                                                      
[14.077848219550134, 'False', 0.09743533465526928]                                                                     
,                                                                                                                      
[1.2621825221705736, 'True', 0.04786393041368967]                                                                      
,                                                                                                                      
[10.989845200610606, 'True', 0.10332305374816148]                                                                      
,                                                                                                                      
[17.76171887291151, 'True', 0.09244069912242542]                                                                       
,                                                                                                                      
[12.026712864441981, 'False', 0.09814759624274086]                                                                     
,                                                                                                                      
[10.479238216346905, 'True', 0.10169209118467032]                                                                      
,                                                                                                                      
[7.1638965003614725, 'True', 0.10213390395266272]                                                                      
,                                                                                                                      
[23.114824969919063, 'True', 0.08410790969399054]                                                                      
,                                                                                                                      
[31.220184618373096, 'True', 0.0827363073706658]                                                                       
,                                                                                                                      
[19.811006908518713, 'False', 0.08959108826272269]                                                                     
,                                                                                                                      
[26.888914338330196, 'True', 0.08835809541838505]                                                                      
,                                                                                                                      
[54.978496520886154, 'True', 0.0751023000810319]                                                                       
,                                                                                                                      
[4.277539571812326, 'True', 0.09248045567998679]                                                                       
,                                                                                                                      
[64.41762693619319, 'False', 0.0697870129622364]                                                                       
,                                                                                                                      
[42.47125375467726, 'True', 0.08338593589048247]                                                                       
,                                                                                                                      
[10.411566010787821, 'True', 0.09672877937182181]                                                                      
,                                                                                                                      
[36.63770843853095, 'True', 0.08507275828823253]                                                                       
,                                                                                                                      
[46.78989540109957, 'True', 0.07680086390782279]                                                                       
,                                                                                                                      
[51.8139109012666, 'False', 0.0764759157481214]                                                                        
,                                                                                                                      
[17.0276972671206, 'True', 0.09075979017438544]                                                                        
,                                                                                                                      
[23.552065234697473, 'True', 0.08919747855875794]                                                                      
,                                                                                                                      
[29.937301814327306, 'True', 0.08399690162150807] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','MRR']
metric

,neighbours,center,MRR
0,58.107511,False,0.071968
1,53.402378,True,0.075958
2,21.842746,True,0.088775
3,83.931522,False,0.068922
4,99.311226,True,0.062309
...,...,...,...
95,46.789895,True,0.076801
96,51.813911,False,0.076476
97,17.027697,True,0.090760
98,23.552065,True,0.089197


In [6]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,neighbours,center,MRR
63,8.732420,True,0.103785
80,10.989845,True,0.103323
69,10.592690,True,0.103278
84,7.163897,True,0.102134
73,14.394287,True,0.102099
23,8.515980,True,0.101762
83,10.479238,True,0.101692
26,8.760154,True,0.101649
29,9.113039,True,0.101476
35,8.233196,True,0.100721


In [7]:
metric.to_csv('metric_MRR.csv', index=False)